# day 3-1

このノートブックの実行例は[こちら(HTML版)](https://drive.google.com/file/d/1fB1D3tbmcAJLwd57XEqKrx6gvTO1PbhM/view?usp=drive_link)で確認できます

---

## 0. 準備

### 0.1 必要なパッケージのインストール

In [ ]:
%%capture
!sudo apt-get update
!sudo apt-get install -y automake autoconf perl libtool graphviz libgraphviz-dev
!pip install -U japanize_matplotlib pyvis pygraphviz mca

教材のダウンロード

In [ ]:
!git clone https://github.com/haradatm/lecture-gssm2025

### 0.2 MeCab インストール (時間目安: 約3分)

In [ ]:
%%time
!bash lecture-gssm2025/scripts/install_mecab.sh >> install_mecab.log 2>&1
!tail -n 1 install_mecab.log

### 0.3 CaboCha インストール (時間目安: 約4分)

In [ ]:
%%time
!bash lecture-gssm2025/scripts/install_cabocha.sh >> install_cabocha.log 2>&1
!tail -n 1 install_cabocha.log

### 0.4 セッションの再起動

In [ ]:
import os
os.kill(os.getpid(), 9)

セッションの再起動後は,以下のセルから実行してください. **注意: これより前のセルを再度実行する必要はありません!**

---

### 0.5 動作確認

In [ ]:
import MeCab
tagger = MeCab.Tagger("-r ./tools/usr/local/etc/mecabrc")
print(tagger.parse("今日はいい天気です"))

In [ ]:
import CaboCha
cp = CaboCha.Parser("-r ./tools/usr/local/etc/cabocharc")
tree = cp.parse("今日はいい天気です")
print(tree.toString(CaboCha.FORMAT_LATTICE))



---



## 1. テキスト解析 (1)

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random
import numpy as np

# 再現性のために乱数を固定する
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)

### 1.1 形態素解析

(1) そのまま実行する

In [ ]:
import MeCab
tagger = MeCab.Tagger("-r ./tools/usr/local/etc/mecabrc")
print(tagger.parse("今日はいい天気です"))

(2) 扱いやすいように Pandas の DataFrame に格納する

In [ ]:
import pandas as pd

node = tagger.parseToNode("今日はいい天気です")
features = []
while node:
    features.append(node.feature.split(','))
    node = node.next

columns = [
    "品詞",
    "品詞細分類1",
    "品詞細分類2",
    "品詞細分類3",
    "活用型",
    "活用形",
    "基本形",
    "読み",
    "発音",
]
pd.DataFrame(features, columns=columns)

### 1.2 係り受け解析

(1) そのまま実行する

In [ ]:
import CaboCha
cp = CaboCha.Parser("-r ./tools/usr/local/etc/cabocharc")
tree = cp.parse("今日はいい天気です")
print(tree.toString(CaboCha.FORMAT_LATTICE))

(2) ツリー形式で出力する

In [ ]:
print(cp.parseToString("今日はいい天気です"))

(3) 係り受けペアを出力する

In [ ]:
# 構文木(tree)からチャンクを取り出す
def get_chunks(tree):
    chunks = {}
    key = 0
    for i in range(tree.size()):
        tok = tree.token(i)
        if tok.chunk:
            chunks[key] = tok.chunk
            key += 1
    return chunks


# チャンク(chunk)から表層形を取り出す
def get_surface(chunk):
    surface = ""
    beg = chunk.token_pos
    end = chunk.token_pos + chunk.token_size
    for i in range(beg, end):
        token = tree.token(i)
        surface += token.surface
    return surface

In [ ]:
tree = cp.parse("今日はいい天気です")
chunks = get_chunks(tree)

for from_chunk in chunks.values():
    if from_chunk.link < 0:
        continue
    to_chunk = chunks[from_chunk.link]

    from_surface = get_surface(from_chunk)
    to_surface = get_surface(to_chunk)

    print(from_surface, '->', to_surface)

### 1.3 辞書追加

(1) 辞書追加前に MeCab の解析結果を確認する

In [ ]:
import MeCab

tagger = MeCab.Tagger("-r ./tools/usr/local/etc/mecabrc")
print(tagger.parse("この泉質は極上です"))
print(tagger.parse("この海鮮丼は美味しいです"))
print(tagger.parse("近くにスカイツリーがあります"))
print(tagger.parse("浴室にバスタオルがありません"))

(2) 辞書追加前に CaboCha の解析結果を確認する

In [ ]:
import CaboCha

cp = CaboCha.Parser("-r ./tools/usr/local/etc/cabocharc")
print(cp.parse("この泉質は極上です").toString(CaboCha.FORMAT_LATTICE))
print(cp.parse("この海鮮丼は美味しいです").toString(CaboCha.FORMAT_LATTICE))
print(cp.parse("近くにスカイツリーがあります").toString(CaboCha.FORMAT_LATTICE))
print(cp.parse("浴室にバスタオルがありません").toString(CaboCha.FORMAT_LATTICE))

(3) 追加したい形態素の情報を CSV ファイル(user_dic.csv)に追記する

In [ ]:
!echo '"泉質",-1,-1,1,名詞,一般,*,*,*,*,泉質,センシツ,センシツ,USER"' >> ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv
!echo '"海鮮丼",-1,-1,1,名詞,一般,*,*,*,*,海鮮丼,カイセンドン,カイセンドン,USER"' >> ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv
!echo '"スカイツリー",-1,-1,1,名詞,一般,*,*,*,*,スカイツリー,スカイツリー,スカイツリー,USER"' >> ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv
!echo '"バスタオル",-1,-1,1,名詞,一般,*,*,*,*,バスタオル,バスタオル,バスタオル,USER"' >> ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv
!cat ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv

(4) CSVファイル(user_dic.csv)をコンパイルして辞書(user.dic)を作成する

In [ ]:
# !./tools/usr/local/libexec/mecab/mecab-dict-index \
# -d ./tools/usr/local/lib/mecab/dic/ipadic \
# -u ./tools/usr/local/lib/mecab/dic/ipadic/user.dic \
# -f utf-8 -t utf-8 \
# ./tools/usr/local/lib/mecab/dic/ipadic/user_dic.csv

(5) 辞書追加後に MeCab の解析結果を確認する

In [ ]:
import MeCab

tagger = MeCab.Tagger("-r ./tools/usr/local/etc/mecabrc")
print(tagger.parse("この泉質は極上です"))
print(tagger.parse("この海鮮丼は美味しいです"))
print(tagger.parse("近くにスカイツリーがあります"))
print(tagger.parse("浴室にバスタオルがありません"))

(6) 辞書追加後に CaboCha の解析結果を確認する

In [ ]:
import CaboCha

cp = CaboCha.Parser("-r ./tools/usr/local/etc/cabocharc")
print(cp.parse("この泉質は極上です").toString(CaboCha.FORMAT_LATTICE))
print(cp.parse("この海鮮丼は美味しいです").toString(CaboCha.FORMAT_LATTICE))
print(cp.parse("近くにスカイツリーがあります").toString(CaboCha.FORMAT_LATTICE))
print(cp.parse("浴室にバスタオルがありません").toString(CaboCha.FORMAT_LATTICE))



---



## Appendix. データ理解 (Python の場合)

### A.1 データのダウンロード

以下のデータをダウンロードします

| ファイル名 | 件数 | データセット | 備考 |
| --- | --- | --- | --- |
| rakuten-1000-2024-2025.xlsx.zip | 10,000 | •レジャー+ビジネスの 10エリア<br>•エリアごと 1,000件 (ランダムサンプリング)<br>•期間: 2024/1~2025 GW明け | 本講義の全体を通して使用する |

In [ ]:
# rakuten-1000-2024-2025.xlsx.zip をダウンロードする
FILE_ID = "1yKabZ7qJMRrIrP4Vtq-RrSZAqFsZriQS"
!gdown {FILE_ID}
!unzip -o rakuten-1000-2024-2025.xlsx.zip

### A.2 データの読み込み (DataFrame型)

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_excel("rakuten-1000-2024-2025.xlsx")
print(df.shape)
display(df.head())

### A.3 集計

#### (1) エリア別の件数を表示する

In [ ]:
display(df.pivot_table(index=['カテゴリー','エリア'], columns=None, values='コメント', aggfunc='count'))

サンプリングデータなので、すべて1000件になっていることを確認する

#### (2) 投稿者の傾向 (年代別x性別、全体で100%)

In [ ]:
cross_df = pd.crosstab(df['年代'], df['性別'], margins=True, margins_name='合計', normalize=True) * 100
display(cross_df.loc[:,['男性','女性','na','合計']].style.format('{:.2f}%').bar(axis=None, vmin=0, width=90, height=90, align='left'))

`na` を除いて表示する

In [ ]:
cross_df = pd.crosstab(df['年代'], df['性別'], margins=False, normalize=True) * 100
display(cross_df.loc[cross_df.index!='na',['男性','女性']].style.format('{:.2f}%').bar(axis=None, vmin=0, width=90, height=90, align='left'))

#### (3) 投稿者の傾向 (性別xカテゴリ別、列ごとで100%)

In [ ]:
cross_df = pd.crosstab(df['性別'], df['カテゴリー'], margins=True, margins_name='合計', normalize='columns') * 100
display(cross_df.loc[['男性','女性','na'],:].style.format('{:.2f}%').bar(axis=None, width=90, height=90, align='left'))

`na` を除いて表示する

In [ ]:
cross_df = pd.crosstab(df['性別'], df['カテゴリー'], margins=False, normalize='columns') * 100
display(cross_df.loc[['男性','女性'],:].style.format('{:.2f}%').bar(axis=None, vmin=0, width=90, height=90, align='left'))

#### (4) 投稿者の傾向 (性別xエリア別、列ごとで100%)

In [ ]:
cross_df = pd.crosstab(df['性別'], [df['カテゴリー'],df['エリア']], margins=True, margins_name='合計', normalize='columns') * 100
display(cross_df.loc[['男性','女性','na'],:].style.format('{:.2f}%'))

カラーバーにすると横に長くなるため、ヒートマップでプロットする

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.figure(figsize=(10,3))
sns.heatmap(cross_df.loc[['男性','女性','na'],:], annot=True, fmt='.2f', cmap='Blues')
plt.show()

`na` を除いてプロットする

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.figure(figsize=(10,2))
sns.heatmap(cross_df.loc[['男性','女性'],:], annot=True, fmt='.2f', cmap='Blues')
plt.show()

#### (5) 投稿者の傾向 (年代xエリア別、列ごとで100%)

In [ ]:
cross_df = pd.crosstab(df['年代'], [df['カテゴリー'],df['エリア']], margins=True, margins_name='合計', normalize='columns') * 100
display(cross_df.style.format('{:.2f}%'))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.figure(figsize=(10,3))
sns.heatmap(cross_df, annot=True, fmt='.2f', cmap='Blues')
plt.show()

`na` を除いてプロットする

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.figure(figsize=(10,3))
sns.heatmap(cross_df.loc[cross_df.index!='na',:], annot=True, fmt='.2f', cmap='Blues')
plt.show()

#### (6) 投稿者の傾向 (同伴者別xエリア別、列ごとで100%)

In [ ]:
cross_df = pd.crosstab(df['同伴者'], [df['カテゴリー'],df['エリア']], margins=True, margins_name='合計', normalize='columns') * 100
display(cross_df.loc[['一人','家族','恋人','友達','仕事仲間','その他'],:].style.format('{:.2f}%'))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.figure(figsize=(10,3))
sns.heatmap(cross_df.loc[['一人','家族','恋人','友達','仕事仲間','その他'],:], annot=True, fmt='.2f', cmap='Blues')
plt.show()

#### (7) 数値評価の構成 (総合別xカテゴリ-エリア別、列ごとに100%)

In [ ]:
cross_df = pd.crosstab(df['総合'], [df['カテゴリー'],df['エリア']], margins=True, margins_name='合計', normalize='columns') * 100
display(cross_df.loc[[5,4,3,2,1],:].style.format('{:.2f}%'))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.figure(figsize=(10,3))
sns.heatmap(cross_df.loc[[5,4,3,2,1],:], annot=True, fmt='.2f', cmap='Blues')
plt.show()

#### (8) 数値評価の平均 (カテゴリ-エリア別x数値評価別)

カテゴリ別

In [ ]:
pivot_df = df.pivot_table(index=['カテゴリー','エリア'], values=['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合'], margins=False, aggfunc='mean', dropna=True)
display(pivot_df.loc[:,['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合']].style.format('{:.2f}').background_gradient(axis=None))

エリア別

In [ ]:
pivot_df = df.pivot_table(index=['カテゴリー'], values=['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合'], margins=False, aggfunc='mean', dropna=True)
display(pivot_df.loc[:,['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合']].style.format('{:.2f}').background_gradient(axis=None))

#### (9)数値評価の平均 (年代x性別)

20~30代

In [ ]:
pivot_df = df[df['年代'].isin(['20代','30代'])].pivot_table(index=['カテゴリー','性別'], values=['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合'], margins=False, aggfunc='mean', dropna=True)
display(pivot_df.loc[(['A_レジャー','B_ビジネス'],['男性','女性']),['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合']].style.format('{:.2f}').background_gradient(axis=None))

40~50代

In [ ]:
pivot_df = df[df['年代'].isin(['40代','50代'])].pivot_table(index=['カテゴリー','性別'], values=['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合'], margins=False, aggfunc='mean', dropna=True)
display(pivot_df.loc[(['A_レジャー','B_ビジネス'],['男性','女性']),['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合']].style.format('{:.2f}').background_gradient(axis=None))

60~80代

In [ ]:
pivot_df = df[df['年代'].isin(['60代','70代','80代'])].pivot_table(index=['カテゴリー','性別'], values=['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合'], margins=False, aggfunc='mean', dropna=True)
display(pivot_df.loc[(['A_レジャー','B_ビジネス'],['男性','女性']),['サービス','立地','部屋','設備・アメニティ','風呂','食事','総合']].style.format('{:.2f}').background_gradient(axis=None))